In [1]:
import numpy as np

##### *Creating a class that would make the calculations easier*

In [2]:
class Process:
    def __init__(self, lamb, duration, mu, arrival_times=None,
                 service_times=None, number_of_customers=0):
        """
        Args:
            lamb:                 Parameter lambda of the poisson distribution
            duration:             Duration of the poisson process (T)
            mu:                   Parameter mu of the exponentially distributed service times
            arrival_times:        Inter-arrival times of the customers
            service_times:        Service times of the customers
            number_of_customers:  Number of customers in the system
        """
        self.lamb = lamb
        self.duration = duration
        self.mu = mu
        self.number_of_customers = number_of_customers
        self.service_times = service_times
        self.arrival_times = arrival_times
        self.arrivals_services()
        
    def arrivals_services(self):
        random_number = int(np.random.poisson(self.lamb * self.duration  * 1.3))
        inter_arrivals = np.random.exponential(1/self.lamb, size = random_number)
        arrival_times = np.cumsum(inter_arrivals)
        self.arrival_times = list(arrival_times[arrival_times <= self.duration])
        self.number_of_customers = len(self.arrival_times)
        self.service_times = list(np.random.exponential(self.mu, size=self.number_of_customers))
    
    def simulate_mmk(self, k: int) -> list:
        """
        Calculates the waiting time for M/M/k Queue
        Args:
            k: the number of servers available
        """
        assert len(self.arrival_times) == len(self.service_times) #Just to make sure
        
        waiting_times = []
        next_idle_times =  np.zeros(k)
        for i in range(self.number_of_customers):
            earliest_finish_time = min(next_idle_times)
            index_server = np.argmin(next_idle_times)
            new_start_time = max(earliest_finish_time, self.arrival_times[i])
            new_wait_time = max(0, new_start_time - self.arrival_times[i])
            new_finish_time = new_start_time + self.service_times[i]
            next_idle_times[index_server] = new_finish_time
            waiting_times.append(new_wait_time)
        return waiting_times
        
    def calculate_mean_waiting(self, waiting: list, limit: list):
        """
        Args:
            waiting: waiting times of the customers
            limit: 2 element list. First element is the start and second element is the end of the interval

        Returns: Mean waiting time for the given interval by 'limit'
        """
        corresponding_waiting_times = self.get_waiting_times(waiting, limit)
        return np.mean(corresponding_waiting_times)

    def get_waiting_times(self, waiting: list, limit=None):
        """
        Args:
            waiting: waiting times of the customers
            limit: 2 element list. First element is the start and second element is the end of the interval

        Returns: The waiting times for the given limit.
                    If limit=None, Then all waiting times are returned for the duration T
        """
        if limit is None:
            lower_bound = 0
            upper_bound = self.duration
        else:
            lower_bound = limit[0]
            upper_bound = limit[1]
        interval = [num for num in self.arrival_times if lower_bound <= num <= upper_bound]
        lower_index= self.arrival_times.index(interval[0])
        upper_index = self.arrival_times.index(interval[len(interval) - 1])
        desired_waiting_times = waiting[lower_index:upper_index+1]
        return desired_waiting_times

    def get_arrival_times(self, limit=None):
        """
        Args:
            limit: 2 element list. First element is the start and second element is the end of the interval

        Returns: Arrival times for customers in the given interval
        """
        if limit is None:
            lower_bound = 0
            upper_bound = self.duration
        else:
            lower_bound = limit[0]
            upper_bound = limit[1]
        interval = [num for num in self.arrival_times if lower_bound <= num <= upper_bound]
        return interval


##### *Setting up the right parameters for the simulation*

$\lambda = 2$ customer per minute

$ E[S_i] = 35 $ seconds $= \dfrac{7}{12}$ minutes

Duration = $120$ minutes

##### *Question i)*

In [3]:
n = 50

In [4]:
means = []
for i in range(n):
    food_truck = Process(lamb=2, duration=120, mu=7/12)
    waitings = food_truck.simulate_mmk(k=1)
    mean = food_truck.calculate_mean_waiting(waitings, [15,30]) #11:15-11:30am corresponds to 15-30m if we take 11am as 0m.
    means.append(mean)
np.mean(means)

5.955433590961307

##### *Question ii)*

In [5]:
means = []
for i in range(n):
    food_truck = Process(lamb=2, duration=120, mu=7/12)
    waitings = food_truck.simulate_mmk(k=1)
    mean = food_truck.calculate_mean_waiting(waitings, [105, 120]) #12:45-1:00pm corresponds to 105-120m if we take 11am as 0m.
    means.append(mean)
np.mean(means)

21.545003883619597

##### *Question iii)*

The average waiting time for customers who arrive at food truck from 12:45 PM to 1:00 PM is larger than
the average waiting time for customers who arrive at food truck from 11:15 AM to 11:30 AM. We found this logical
and intuitive since the rate of serving customers is 1 in 35 seconds. On the other hand, the rate of customers coming to the
food truck is 1 in 30 seconds. Because of the fact that the rate of serving customers is higher than the rate of
customer arrivals, customers will be accumulated in the queue by the time. Therefore, in later hours, average
waiting time would be higher as compare to average waiting time in the early service starting times.


##### *Question iv)*

In [6]:
values=[]
n = 50
for i in range(n):
    food_truck = Process(lamb=2, duration=120, mu=7/12)
    waitings = food_truck.simulate_mmk(1)
    desired_interval = food_truck.get_waiting_times(waitings, limit=[15,30])
    actual_length = len(desired_interval)
    impatience_length = len([num for num in desired_interval if num > 3])
    percentage = (100*impatience_length) / actual_length
    values.append(percentage)
first = np.mean(values)

values=[]
for i in range(n):
    food_truck = Process(lamb=2, duration=120, mu=7/12)
    waitings = food_truck.simulate_mmk(1)
    desired_interval = food_truck.get_waiting_times(waitings, limit=[105,120])
    actual_length = len(desired_interval)
    impatience_length = len([num for num in desired_interval if num > 3])
    percentage = (100*impatience_length) / actual_length
    values.append(percentage)
second = np.mean(values)

print('Out of {} trials, '.format(n))
print('The percentage of people waiting more than 3 minutes between 11:15 AM and 11:30 AM is approximately {}%'.format(
    np.round(first, 2)))
print('While the percentage of people waiting more than 3 minutes between 12:45 PM and 1:00 PM is approximately {}%'.format(
    np.round(second, 2)))

Out of 50 trials, 
The percentage of people waiting more than 3 minutes between 11:15 AM and 11:30 AM is approximately 67.72%
While the percentage of people waiting more than 3 minutes between 12:45 PM and 1:00 PM is approximately 96.0%


##### *Question v)*

In [7]:
percentages = []
average_waitings = []

for i in range(n):
    system = Process(2,120,7/12)
    system_waitings = system.simulate_mmk(1)
    arrivals_times = system.get_arrival_times(limit=[105,120])

    customers_left = 0
    customer_in_the_interval = len(arrivals_times)

    for time in arrivals_times:
        index = system.arrival_times.index(time)
        arrivals_before = system.arrival_times[:index]
        waiting_before = system_waitings[:index]
        services_before = system.service_times[:index]
        customers_in_queue = 0

        for j in range(index):
            arrival_time_j = arrivals_before[j]
            waiting_time_j = waiting_before[j]
            service_start_time = arrival_time_j + waiting_time_j
            if service_start_time >= time:
                customers_in_queue += 1

        if customers_in_queue >= 4:
            customers_left += 1
            del system.arrival_times[index]
            del system.service_times[index]
            del system_waitings[index]

    perc = customers_left/customer_in_the_interval*100
    percentages.append(perc)

    interval = [num for num in system.arrival_times if 105 <= num <= 120]
    if len(interval) == 0:
        continue
    else:
        lower_index= system.arrival_times.index(interval[0])
        upper_index = system.arrival_times.index(interval[len(interval) - 1])
        desired_waiting_times = system_waitings[lower_index:upper_index+1]
        mean_waiting = np.mean(desired_waiting_times)
        average_waitings.append(mean_waiting)

np.mean(percentages), perc
print('The percentage of the customers who would leave the system is approximately {}%.'.format(
    np.round(np.mean(percentages))))
print('The mean waiting time for the customers who stayed on the queue is approximately {} minutes.'.format(
    np.round(np.mean(average_waitings),2)))

The percentage of the customers who would leave the system is approximately 92.0%.
The mean waiting time for the customers who stayed on the queue is approximately 11.27 minutes.


##### *Question vi)*

In [8]:
#i)

means = []
for i in range(n):
    food_truck = Process(lamb=2, duration=120, mu=1/2)
    waitings = food_truck.simulate_mmk(1)
    mean = food_truck.calculate_mean_waiting(waitings, [15,30]) #11:15-11:30am corresponds to 15-30m if we take 11am as 0m.
    means.append(mean)
np.mean(means)

3.6165649021645754

In [9]:
#ii)

means = []
for i in range(n):
    food_truck = Process(lamb=2, duration=120, mu=1/2)
    waitings = food_truck.simulate_mmk(1)
    mean = food_truck.calculate_mean_waiting(waitings, [105, 120]) #12:45-1:00pm corresponds to 105-120m if we take 11am as 0m.
    means.append(mean)
np.mean(means)

8.354224311202154

###### iii)

The difference between average waiting time for both intervals is less than the previous version of the question where the average time for a serving was 35 seconds. This happens because, now, the expected serving time of a customer is 1 in 30 seconds, which is exactly equal to the inter-arrival rate of the customers. Therefore, it is logical that these two values in different intervals are more close to each other.

In [10]:
#iv)

values=[]
for i in range(n):
    food_truck = Process(lamb=2, duration=120, mu=1/2)
    waitings = food_truck.simulate_mmk(1)
    desired_interval = food_truck.get_waiting_times(waitings, limit=[15,30])
    actual_length = len(desired_interval)
    impatience_length = len([num for num in desired_interval if num > 3])
    percentage = (100*impatience_length) / actual_length
    values.append(percentage)
first = np.mean(values)

values=[]
for i in range(n):
    food_truck = Process(lamb=2, duration=120, mu=1/2)
    waitings = food_truck.simulate_mmk(1)
    desired_interval = food_truck.get_waiting_times(waitings, limit=[105,120])
    actual_length = len(desired_interval)
    impatience_length = len([num for num in desired_interval if num > 3])
    percentage = (100*impatience_length) / actual_length
    values.append(percentage)
second = np.mean(values)

print('Out of {} trials, '.format(n))
print('The percentage of people waiting more than 3 minutes between 11:15 AM and 11:30 AM is approximately {}%'.format(
    np.round(first, 2)))
print('While the percentage of people waiting more than 3 minutes between 12:45 PM and 1:00 PM is approximately {}%'.format(
    np.round(second, 2)))

Out of 50 trials, 
The percentage of people waiting more than 3 minutes between 11:15 AM and 11:30 AM is approximately 41.41%
While the percentage of people waiting more than 3 minutes between 12:45 PM and 1:00 PM is approximately 70.99%


In [13]:
#v)

percentages = []
average_waitings = []

for i in range(n):
    system = Process(2,120,1/2)
    system_waitings = system.simulate_mmk(1)
    arrivals_times = system.get_arrival_times(limit=[105,120])

    customers_left = 0
    customer_in_the_interval = len(arrivals_times)

    for time in arrivals_times:
        index = system.arrival_times.index(time)
        arrivals_before = system.arrival_times[:index]
        waiting_before = system_waitings[:index]
        services_before = system.service_times[:index]
        customers_in_queue = 0

        for j in range(index):
            arrival_time_j = arrivals_before[j]
            waiting_time_j = waiting_before[j]
            service_start_time = arrival_time_j + waiting_time_j
            if service_start_time >= time:
                customers_in_queue += 1

        if customers_in_queue >= 4:
            customers_left += 1
            del system.arrival_times[index]
            del system.service_times[index]
            del system_waitings[index]

    perc = customers_left/customer_in_the_interval*100
    percentages.append(perc)

    interval = [num for num in system.arrival_times if 105 <= num <= 120]
    if len(interval) == 0:
        continue
    else:
        lower_index= system.arrival_times.index(interval[0])
        upper_index = system.arrival_times.index(interval[len(interval) - 1])
        desired_waiting_times = system_waitings[lower_index:upper_index+1]
        mean_waiting = np.mean(desired_waiting_times)
        average_waitings.append(mean_waiting)

np.mean(percentages), perc
print('The percentage of the customers who would leave the system is approximately {}%.'.format(
    np.round(np.mean(percentages))))
print('The mean waiting time for the customers who stayed on the queue is approximately {} minutes.'.format(
    np.round(np.mean(average_waitings),2)))

The percentage of the customers who would leave the system is approximately 63.0%.
The mean waiting time for the customers who stayed on the queue is approximately 6.24 minutes.
